In [144]:
%load_ext autoreload
%autoreload 2
    
from data import get_dataset, data_to_sequences
from eval_utils import sasrec_model_scoring

from time import time
from functools import reduce
import uuid
from tqdm import tqdm
import os

from clearml import Task, Logger
from omegaconf import OmegaConf
import hydra

import numpy as np
import torch
from torch.utils.data import DataLoader

from model import SASRec
from data import get_dataset, data_to_sequences, SequentialDataset
from utils import topn_recommendations, downvote_seen_items
from eval_utils import model_evaluate, sasrec_model_scoring, get_test_scores

from sklearn.linear_model import LogisticRegression

os.environ['CUDA_VISIBLE_DEVICES'] = '5'
device='cuda'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
_, data_description_temp, testset_valid_temp, _, holdout_valid_temp, _ = get_dataset(splitting='temporal_full', q=0.8)
data_description_temp

Filtered 115 invalid observations.
Filtered 11 invalid observations.
Filtered 4 invalid observations.


{'users': 'userid',
 'items': 'itemid',
 'order': 'timestamp',
 'n_users': 5400,
 'n_items': 3658}

In [141]:
_, data_description_full, testset_valid_full, _, holdout_valid_full, _ = get_dataset(splitting='full', q=0.8)
data_description_full

userid         1783
itemid         3511
rating            5
timestamp    110948
dtype: int64
Filtered 115 invalid observations.
Filtered 11 invalid observations.
Filtered 4 invalid observations.
userid         1783
itemid         3464
rating            5
timestamp    110850
dtype: int64
Filtered 107 invalid observations.
Filtered 8 invalid observations.
Filtered 13 invalid observations.
Filtered 2 invalid observations.


{'users': 'userid',
 'items': 'itemid',
 'order': 'timestamp',
 'n_users': 5400,
 'n_items': 3658}

In [ ]:
SUBSEQS = 
GT_SUBSEQ = 
список_из_логитов_для_каждого_айтема, ембеддинг_стейта_перед_предсказанием = sasrec_model_scoring(model, SUBSEQS, data_description, device=)

логрег(ембеддинг_стейта_перед_предсказанием, гт) валидация? тест? чтоб чекнуть?

In [4]:
base_config = dict(
    num_epochs = 1,
    maxlen = 10,
    hidden_units = 64,
    dropout_rate = 0.3,
    num_blocks = 2,
    num_heads = 1,
    batch_size = 4,
    learning_rate = 1e-3,
    fwd_type = 'ce',
    l2_emb = 0,
    manual_seed = 123,
    sampler_seed = 123,
    patience = 10,
    skip_epochs = 1
)

In [19]:
n_users = data_description['n_users']
n_items = data_description['n_items']
model = SASRec(n_items, base_config).to(device)

model.eval()

SASRec(
  (item_emb): Embedding(3660, 64, padding_idx=3659)
  (pos_emb): Embedding(10, 64)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (attention_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((64,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0-1): 2 x MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
    )
  )
  (forward_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((64,), eps=1e-08, elementwise_affine=True)
  )
  (forward_layers): ModuleList(
    (0-1): 2 x PointWiseFeedForward(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (dropout1): Dropout(p=0.3, inplace=False)
      (relu): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.3, inplace=False)
    )
  )
  (last_layernorm): LayerNorm((64,), eps=1e-08, elementwise_affine=True)
)

In [20]:
model.item_emb.weight

Parameter containing:
tensor([[-0.0164,  0.0013, -0.0199,  ...,  0.0332, -0.0379, -0.0270],
        [-0.0160,  0.0016, -0.0094,  ..., -0.0199,  0.0177,  0.0157],
        [ 0.0112,  0.0317, -0.0162,  ..., -0.0389, -0.0341, -0.0391],
        ...,
        [ 0.0075, -0.0049, -0.0062,  ..., -0.0387, -0.0139, -0.0225],
        [-0.0234,  0.0156,  0.0289,  ..., -0.0089, -0.0065, -0.0019],
        [ 0.0385, -0.0192, -0.0353,  ...,  0.0030, -0.0096, -0.0273]],
       device='cuda:0', requires_grad=True)

In [16]:
seqs1 = data_to_sequences(testset_valid, data_description)

In [24]:
subset = testset_valid[testset_valid.userid < 3]
subset

,userid,itemid,rating,timestamp
0,1,1086,5,978300760
1,1,632,3,978302109
2,1,838,3,978301968
3,1,3136,4,978300275
4,1,2132,5,978824291
...,...,...,...,...
177,2,346,5,978299686
178,2,1135,2,978299200
179,2,1136,5,978299418
180,2,3600,1,978299535


In [106]:
test_sequences = data_to_sequences(subset, data_description)

scores = []
for _, seq in test_sequences.items():
    with torch.no_grad():
        predictions = model.score(torch.tensor(seq, device=device, dtype=torch.long))
    scores.append(predictions.detach().cpu().numpy())
a = np.concatenate(scores, axis=0)
a

array([[0.3327778 , 0.00746917, 0.12461472, ..., 0.22785743, 0.13299876,
        0.10108273],
       [0.3598424 , 0.0702517 , 0.13007008, ..., 0.24712753, 0.04797748,
        0.17051247]], dtype=float32)

In [151]:
_, a = model.score_with_state(torch.tensor(seq, device=device, dtype=torch.long))
a

tensor([ 0.8240,  0.0203, -0.6111, -0.2323,  0.1054, -0.3582, -0.1421, -0.3221,
        -0.4069,  0.9805, -0.3981,  0.9406, -1.2298, -1.2345, -2.0832,  0.3218,
         1.6107, -1.0859, -1.7604,  1.5930, -1.0390,  1.8432, -0.8485,  1.3047,
         0.8276,  0.0060, -0.3096,  1.6431, -0.6786, -1.1276,  0.7542,  0.2431,
        -0.7762,  0.0750, -0.2243,  0.8205, -0.9569, -0.4007, -1.2098, -0.1817,
        -0.2230, -0.6141, -0.1028, -0.5813,  0.9197,  1.6171, -1.5417,  1.8323,
         0.8317, -0.3831,  1.5476,  0.9742, -1.5195,  0.7749, -0.7665, -0.1304,
        -0.0973,  0.9355,  2.4113,  0.6006, -0.6536, -0.9658, -0.8672, -0.2947],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [109]:
len(test_sequences)

2

In [49]:
seq

[1090,
 1102,
 1109,
 2479,
 1183,
 2702,
 1117,
 1086,
 309,
 2783,
 2618,
 1105,
 1739,
 1099,
 573,
 2846,
 3194,
 1668,
 498,
 1001,
 2277,
 2788,
 106,
 1860,
 2898,
 1137,
 2856,
 1241,
 1088,
 1751,
 1747,
 844,
 1631,
 995,
 1756,
 3197,
 3370,
 3450,
 1149,
 1748,
 1595,
 2490,
 1762,
 1014,
 826,
 3178,
 3300,
 2612,
 3067,
 2820,
 258,
 2090,
 570,
 1143,
 2823,
 1134,
 3415,
 1749,
 1135,
 2017,
 3394,
 905,
 1987,
 2049,
 1317,
 2995,
 620,
 228,
 1007,
 1030,
 481,
 1136,
 3600,
 1394,
 1081,
 2173,
 2136,
 2098,
 346,
 2859,
 1155,
 3521,
 569,
 1822,
 2344,
 440,
 2675,
 1456,
 463,
 157,
 370,
 3145,
 2996,
 1286,
 20,
 339,
 1386,
 2130,
 1800,
 2057,
 1251,
 621,
 2204,
 1600,
 1253,
 1407,
 2266,
 1266,
 726,
 2641,
 2858,
 358,
 1607,
 159,
 442,
 425,
 1444,
 2394,
 1531,
 2997,
 693,
 1796,
 1919,
 283,
 92,
 1528,
 417,
 1712]

In [26]:
a.shape

(2, 3660)

In [38]:
predictions[0].argmax()

tensor(700, device='cuda:0')

In [45]:
predictions[0].detach().cpu().numpy()

array([0.3598424 , 0.0702517 , 0.13007008, ..., 0.24712753, 0.04797748,
       0.17051247], dtype=float32)

In [46]:
torch.softmax(predictions[0].detach().cpu(), 0)[700]

tensor(0.0006)

In [51]:
def score_policy_e(at, seqt, model, device):
    
    with torch.no_grad():
        predictions = model.score(torch.tensor(seqt, device=device, dtype=torch.long)).detach().cpu()[0]
    
    proba_at = torch.softmax(predictions, 0)[at].numpy()
    
    return proba_at

In [61]:
score_policy_e(3, seq, model, device='cuda')

array(0.00032329, dtype=float32)

In [65]:
test_seq = seq[:10]
test_seq

[1090, 1102, 1109, 2479, 1183, 2702, 1117, 1086, 309, 2783]

In [73]:
test_seq

[1090, 1102, 1109, 2479, 1183, 2702, 1117, 1086, 309, 2783]

In [134]:
subseqs_h = []
seqt = test_seq.copy()
N = 3

for i in range(len(test_seq)-1):
    at = seqt.pop()
    subseqs_h.append((at, seqt.copy()))
subseqs_h[::-1]

[(1102, [1090]),
 (1109, [1090, 1102]),
 (2479, [1090, 1102, 1109]),
 (1183, [1090, 1102, 1109, 2479]),
 (2702, [1090, 1102, 1109, 2479, 1183]),
 (1117, [1090, 1102, 1109, 2479, 1183, 2702]),
 (1086, [1090, 1102, 1109, 2479, 1183, 2702, 1117]),
 (309, [1090, 1102, 1109, 2479, 1183, 2702, 1117, 1086]),
 (2783, [1090, 1102, 1109, 2479, 1183, 2702, 1117, 1086, 309])]

In [139]:
N = 3
subseqs_h = []
seqt = test_seq.copy()

for i in range(len(test_seq)-1):
    at = seqt.pop()
    subseqs_h.append((at, seqt[-N:]))
subseqs_h[::-1]

[(1102, [1090]),
 (1109, [1090, 1102]),
 (2479, [1090, 1102, 1109]),
 (1183, [1102, 1109, 2479]),
 (2702, [1109, 2479, 1183]),
 (1117, [2479, 1183, 2702]),
 (1086, [1183, 2702, 1117]),
 (309, [2702, 1117, 1086]),
 (2783, [1117, 1086, 309])]

In [268]:
def build_model_D(training_D, model_D_sasrec, n, data_description):
    model_D_sasrec.eval()
    states = []
    actions = []
    
    full_sequences = data_to_sequences(training_D, data_description)
    for _, h in tqdm(full_sequences.items()):

        subseqs_h = []
        seqt = h.copy()
        for i in range(len(h)-1):
            at = seqt.pop()
            subseqs_h.append((at, seqt[-n:]))

        # subseqs_h = []
        # seqt = h.copy()
        # for i in range(1):
        #     at = seqt.pop()
        #     subseqs_h.append((at, seqt[-n:]))

        for at, subseq in subseqs_h:
            with torch.no_grad():
                _, state = model_D_sasrec.score_with_state(torch.tensor(subseq, device=device, dtype=torch.long))
                states.append(state.detach().cpu().numpy())
                actions.append(at)
                
    # должно фититься на всех айтемах (все экшены должны быть в таргете хоть раз)
    model_D = LogisticRegression(multi_class='multinomial', max_iter=1).fit(states, actions) #!!!!!
    
    return model_D

In [269]:
model_D_test = build_model_D(testset_valid_full, model, 3, data_description_full)
model_D_test

6040it [00:15, 399.49it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1, multi_class='multinomial')

In [253]:
testset_valid_full[testset_valid_full.userid < 3]

,userid,itemid,rating,timestamp
0,1,1086,5,978300760
1,1,632,3,978302109
2,1,838,3,978301968
3,1,3136,4,978300275
4,1,2132,5,978824291
...,...,...,...,...
177,2,346,5,978299686
178,2,1135,2,978299200
179,2,1136,5,978299418
180,2,3600,1,978299535


In [270]:
model_D_test.classes_

array([   0,    1,    2, ..., 3653, 3654, 3657])

In [222]:
state = np.array([-0.9155,  0.1566, -0.0902,  0.3246, -1.0450,  1.1859,  0.6490, -0.0042,
          0.2900,  0.0849,  0.0386, -0.7189, -0.5044, -1.4471, -0.9739,  0.8755,
          1.0489, -1.7149, -0.6816,  0.7614, -0.6641,  2.8349, -1.1428,  1.2974,
          0.5513, -0.1746,  0.3071,  0.1533, -0.5535,  0.0373,  0.4324,  0.5614,
          0.0516,  0.0781, -0.3319, -0.1633, -0.8561,  1.7090, -0.2278, -0.4855,
         -0.4979, -0.8764, -0.2759,  0.4210,  1.1624,  1.2836,  0.2431, -0.1973,
         -1.0329,  0.4999,  0.1575, -0.0300, -0.0970, -0.2635, -1.2588, -0.4120,
          0.4372,  1.4692,  3.2986,  1.1218, -0.3660, -2.5685, -1.9295, -1.0226])

In [271]:
model_D_test.predict_proba(state.reshape(1, -1))[0]

array([0.00062306, 0.00054591, 0.00057484, ..., 0.00087445, 0.00067112,
       0.00051937])

In [184]:
testset_valid_full[testset_valid_full.userid < 2].sort_values('timestamp')

,userid,itemid,rating,timestamp
31,1,2935,4,978300019
27,1,1552,4,978300055
37,1,941,5,978300055
22,1,1160,5,978300055
24,1,2117,3,978300103
36,1,1633,5,978300172
3,1,3136,4,978300275
47,1,1099,4,978300719
7,1,2566,5,978300719
21,1,680,3,978300760


In [282]:
def score_policy_D(at, seqt, n, model_D, model_D_sasrec, device):
    with torch.no_grad():
        _, state = model_D_sasrec.score_with_state(torch.tensor(seqt, device=device, dtype=torch.long))

    try:
        proba_gt = model_D.predict_proba(state.detach().cpu().numpy().reshape(1, -1))[0, np.where(model_D.classes_ == at)[0][0]]
    except IndexError:
        at = 0
        proba_gt = model_D.predict_proba(state.detach().cpu().numpy().reshape(1, -1))[0, at]

    # proba_gt = model_D.predict_proba(state.detach().cpu().numpy().reshape(1, -1))[0, np.where(model_D.classes_ == at)[0][0]]

    
    return proba_gt

In [283]:
def prod_score_subseqs(h, model_D, model_D_sasrec, model_e, n, device):

    # create subseqs with gts
    subseqs_h = []
    seqt = h.copy()
    for i in range(len(h)-1):
        at = seqt.pop()
        subseqs_h.append((at, seqt.copy()))
        
    probas_e = []
    probas_D = []
    for at, seqt in subseqs_h[::-1]:
        probas_e.append(score_policy_e(at, seqt, model_e, device))
        probas_D.append(score_policy_D(at, seqt[-n:], n, model_D, model_D_sasrec, device))
    
    product_one_h = np.divide(probas_e, probas_D).prod() # prod p_e/p_D

    return product_one_h

def RISEstimate(testset, model_D_sasrec, model_D, model_e, data_description, n=3, gamma=0.99, device='cuda'):
    
    model_D_sasrec.eval() 
    model_e.eval()

    test_sequences = data_to_sequences(testset, data_description)
    m = len(test_sequences)
    ris = 0
    
    for _, h in tqdm(test_sequences.items()):
    
        product_one_h = prod_score_subseqs(h, model_D, model_D_sasrec, model_e, n, device)
        g = np.sum([gamma**t * 1 for t in range(len(h)-1)])

        ris += g * product_one_h
    
    return ris / m

In [248]:
test_sequences

userid
1    [2935, 1160, 1552, 941, 2117, 1633, 3136, 2566...
2    [1090, 1102, 1109, 2479, 1183, 2702, 1117, 108...
Name: itemid, dtype: object

In [278]:
model_D_test.classes_

array([   0,    1,    2, ..., 3653, 3654, 3657])

In [284]:
RISEstimate(testset_valid_temp[testset_valid_temp.userid < 3], model, model_D_test, model, data_description)

3it [00:22,  7.65s/it]


8.407407354049367e-11

In [92]:
score_subseq(test_seq, model)

array([0.00024171, 0.00032   , 0.00026749, 0.00021956, 0.00029772,
       0.00022192, 0.00031178, 0.00029008, 0.0003517 ], dtype=float32)

In [97]:
np.divide([1,2,3], [2,4,6]).prod()

0.125

In [105]:
np.sum([0.99**t * 1 for t in range(len(test_seq)-1)])

8.64827525163591

In [104]:
sum([0.99**t * 1 for t in range(len(test_seq)-1)])

8.64827525163591